In [ ]:
import ipywidgets as widgets
import os
import io
import time
from numpy import array
from PIL import Image, ImageFilter

In [ ]:
BASE_IMG_SIZE = (640, 640)
ROTATION_STEP = 5

In [ ]:
base_img_dir = "images"

In [ ]:
img_files_list = os.listdir(base_img_dir)

In [ ]:
base_imgs = [Image.open(base_img_dir + "/" + file) for file in img_files_list]

In [ ]:
print("Select file...")
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
out = widgets.Output()

In [ ]:
@out.capture()
def on_upload_change(change):
    find_original(Image.open(io.BytesIO(btn_upload.data[-1])))

In [ ]:
btn_upload.observe(on_upload_change, names='_counter')

In [ ]:
 def edges_similarity(img1 ,img2):
    img1_edges = img1.filter(ImageFilter.FIND_EDGES)
    img2_edges = img2.filter(ImageFilter.FIND_EDGES)
    
    #display(img1_edges)
    #display(img2_edges)
    
    img1_edges_matrix = (array(img1_edges)[:, :, 0]) / 255. 
    img2_edges_matrix = (array(img2_edges)[:, :, 0]) / 255.
    return abs(img1_edges_matrix.sum() - img2_edges_matrix.sum())

In [ ]:
def find_original(input_img):
    print ("Calculating...\n")
    
    top_left = tuple(map(lambda base_coord, input_coord: (base_coord - input_coord) / 2, BASE_IMG_SIZE, input_img.size))
    bot_right = tuple(map(lambda top_left_coord, shift: (top_left_coord + shift), top_left, input_img.size))
    
    top_similarities = []
    for img in base_imgs:
        croped_img  = img.crop(top_left + bot_right)
        rotated_imgs = [img.rotate(degree, resample=Image.BILINEAR, fillcolor="white") for degree in range(0, 360, ROTATION_STEP)]
        croped_rotated_imgs = [rotated_img.crop(top_left + bot_right) for rotated_img in rotated_imgs]
        similarities = [edges_similarity(input_img, croped_rotated_img) for croped_rotated_img in croped_rotated_imgs]
        top_similarities.append(min(similarities))
        
    best_match = top_similarities.index(min(top_similarities))
    
    print(f"Original image name is '{img_files_list[best_match]}'\n\n")


In [ ]:
out